**IMPORTING LIBRARIES**

Importing various libraries and packages that we would be needing in our project.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**DATASET READING AND PREPROCESSING**

Dataset contains real and fake videos which will be used for training the model.

Data preprocessing will be in following steps :
    
    1. Split the videos into frames
    2. Detect faces in the frames
    3. Crop the detected faces to focus the model on facial features

In [2]:
import cv2
import os
import dlib

# MENTION THE DIRECTORIES FOR REAL AND FAKE VIDEOS
rvideos_dir = r"F:\Niranjan\abc\real"    # REAL VIDEOS
fvideos_dir = r"F:\Niranjan\abc\fake"    # FAKE VIDEOS

# DIRECTORIES WHERE PREPROCESSED FRAMES FROM REAL AND FAKE VIDEOS WILL BE SAVED
output_real_dir = r"F:\Niranjan\ansh\real"
output_fake_dir = r"F:\Niranjan\ansh\fake"

# ENSURE OUTPUT DIRECTORIES EXIST
os.makedirs(output_real_dir, exist_ok=True)
os.makedirs(output_fake_dir, exist_ok=True)

# LOAD PRETRAINED FACE DETECTOR
detector = dlib.get_frontal_face_detector()

# FUNCTION TO EXTRACT FRAMES AND PROCESS THEM (FACE DETECTION + CROPPING)
def pro_video(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect Faces
        faces = detector(frame, 1)

        # If faces are detected then crop and save them
        for i, face in enumerate(faces):
            x = face.left()
            y = face.top()
            w = face.width()
            h = face.height()

            cropped_face = frame[y:y+h, x:x+w]

            # Saving the cropped face as an image
            output_path = os.path.join(output_dir, f"{os.path.basename(video_path)}_frame_{frame_count}_face_{i}.jpg")
            cv2.imwrite(output_path, cropped_face)

        frame_count += 1

    cap.release()
    print(f"Processed video {video_path}")

# PROCESS ALL REAL VIDEOS
for filename in os.listdir(rvideos_dir):
    if filename.lower().endswith(".mp4"):
        video_path = os.path.join(rvideos_dir, filename)
        pro_video(video_path, output_real_dir)

# PROCESS ALL FAKE VIDEOS
for filename in os.listdir(fvideos_dir):
    if filename.lower().endswith(".mp4"):
        video_path = os.path.join(fvideos_dir, filename)
        pro_video(video_path, output_fake_dir)

Processed video F:\Niranjan\abc\real\01__hugging_happy.mp4
Processed video F:\Niranjan\abc\real\01__outside_talking_still_laughing.mp4
Processed video F:\Niranjan\abc\real\01__podium_speech_happy.mp4
Processed video F:\Niranjan\abc\real\01__talking_against_wall.mp4
Processed video F:\Niranjan\abc\real\01__walk_down_hall_angry.mp4
Processed video F:\Niranjan\abc\fake\01_02__outside_talking_still_laughing__YVGY8LOK.mp4
Processed video F:\Niranjan\abc\fake\01_02__walk_down_hall_angry__YVGY8LOK.mp4
Processed video F:\Niranjan\abc\fake\01_03__hugging_happy__ISF9SP4G.mp4
Processed video F:\Niranjan\abc\fake\01_03__podium_speech_happy__480LQD1C.mp4
Processed video F:\Niranjan\abc\fake\01_03__talking_against_wall__JZUXXFRB.mp4


**MODEL TRAINING**

For training the model we will be using ResNet50

ResNet50 is basically a pre-tarined model used for image classification.

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
# Directories for training data (containing frames from real and fake videos)
train_dir = "F:\\Niranjan\\ansh"

img_size = (224, 224)  # ResNet50 expects this size
batch_size = 32
num_classes = 2   # Real and Fake

# Data augmentation and normalization
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 6897 images belonging to 2 classes.


In [5]:
# Load ResNet50 with pre-trained ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # Fully connected layer
predictions = Dense(num_classes, activation='softmax')(x)  # Output layer for classification (True/False)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze ResNet50 base model layers
for layer in base_model.layers:
  layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# TRAIN THE MODEL

epochs = 10
model.fit(
    train_generator,
    epochs = epochs
)

Epoch 1/10


d:\python\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


216/216 ━━━━━━━━━━━━━━━━━━━━ 342s 2s/step - accuracy: 0.5030 - loss: 0.7622
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.5179 - loss: 0.7021
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.5239 - loss: 0.7039
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 373s 2s/step - accuracy: 0.5254 - loss: 0.6954
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 429s 2s/step - accuracy: 0.5419 - loss: 0.6904
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 372s 2s/step - accuracy: 0.5183 - loss: 0.6908
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 376s 2s/step - accuracy: 0.5355 - loss: 0.6885
Epoch 8/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 363s 2s/step - accuracy: 0.5443 - loss: 0.6872
Epoch 9/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 395s 2s/step - accuracy: 0.5515 - loss: 0.6882
Epoch 10/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 364s 2s/step - accuracy: 0.5310 - loss: 0.6881


In [8]:
# SAVE THE MODEL
model.save('F:\\Niranjan\\model.h5')

**MODEL EVALUATION**

In [10]:
# Import the load_model function from keras.models
from keras.models import load_model

# Load model
model = load_model('F:\\Niranjan\\model.h5')

# Evaluate train data
test_loss, test_accuracy = model.evaluate(train_generator)
print(f"Train Loss: {test_loss}")
print(f"Train Accuracy: {test_accuracy}")

216/216 ━━━━━━━━━━━━━━━━━━━━ 406s 2s/step - accuracy: 0.6136 - loss: 0.6846
Train Loss: 0.6846863627433777
Train Accuracy: 0.6122952103614807


In [12]:
import os

def preprocess_frame(frame_path):
    try:
        img = image.load_img(frame_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        return img_array
    except PermissionError as e:
        print(f"Permission denied: {e}")
        print(f"Current working directory: {os.getcwd()}")
        print(f"File path: {frame_path}")
        return None

In [14]:
# Load the trained model (replace 'model.h5' with your model's path)
model = load_model('F:\\Niranjan\\model.h5')

In [1]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Function to extract frames from video
def extract_frames(video_path, frame_rate=1):
    frames = []
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval = int(fps / frame_rate)  # Extract one frame per second by default
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) % interval == 0:
            frame = cv2.resize(frame, (224, 224))  # Resizing frames to 224x224 for ResNet50 input
            frames.append(frame)
    
    cap.release()
    return np.array(frames)

# Function to preprocess frames for ResNet50
def preprocess_frames(frames):
    frames = frames.astype('float32') / 255.0  # Normalize to [0, 1]
    return frames

# Function to predict whether the video is real or fake
def predict_deepfake(frames, model):
    predictions = model.predict(frames)
    average_prediction = np.mean(predictions)  # Averaging predictions across frames
    return average_prediction

# Load the trained ResNet50 model
model = load_model('F:\\Niranjan\\model.h5')

# Step 1: User inputs the video file path
video_path = input("Please enter the path to the video file (either real or fake): ")

# Step 2: Extract and preprocess frames from the user's video
video_frames = extract_frames(video_path)
video_frames = preprocess_frames(video_frames)

# Step 3: Predict the video's authenticity (real or fake)
video_prediction = predict_deepfake(video_frames, model)

# Step 4: Print the prediction result
print(f'Prediction for the input video: {video_prediction}')

# Step 5: Interpretation of the result (assuming a threshold of 0.5)
threshold = 0.5
if video_prediction < threshold:
    print("The video is classified as REAL.")
else:
    print("The video is classified as FAKE.")


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 912ms/step
Prediction for the input video: 0.4999999403953552
The video is classified as REAL.
